In [38]:
from notebookutils import mssparkutils
from datetime import datetime

In [39]:
start_time = datetime.now()

In [40]:
instance = InstanceId = instanceId
ApiUrl = apiUrl
SchoolYear = schoolYear
DistrictId = DistrictID = districtID = districtId
apiLimit = batchLimit

prepareSAPMetaData = prepareSAPMetadata
zone = submissionsType = sap_pipeline
submissionPeriod = sap_pipelineType

etl_processing = etlProcessing #= True

In [41]:
import random
import string

characters = string.ascii_letters + string.digits
ten_digit_alphanumeric = ''.join(random.choice(characters) for _ in range(10))

pipelineExecutionId = executionId = f'TEST_{ten_digit_alphanumeric}'
pipelineExecutionId

In [43]:
# FIXME: Do not pass uncessary params
input_params = {
        "kvName": kvName,
        "kVName": kvName,
        "workspace": workspace,
        "apiUrl": apiUrl,
        "ApiUrl": apiUrl,
        "instanceId": instanceId,
        "InstanceId": instanceId,
        "moduleName": moduleName,
        "apiLimit": apiLimit,
        "minChangeVer": minChangeVer,
        "maxChangeVer": maxChangeVer,
        "sapVersion": sapVersion,
        "prepareSAPMetaData": prepareSAPMetaData,
        "prepareSAPMetadata": prepareSAPMetaData,
        "submissions": submissions,
        "submissionsType": submissionsType,
        "schoolYear": schoolYear,
        "SchoolYear": schoolYear,
        "districtId": districtId,
        "districtID": districtId,
        "DistrictId": districtId,
        "edfi_url": apiUrl,
        "pipelineExecutionId" : pipelineExecutionId,
        "batchLimit": apiLimit,
        "etlProcessing": etlProcessing,
        "submissionPeriod": submissionPeriod,
        "kvSecret_clientId": kvSecret_clientId,
        "kvSecret_clientSecret": kvSecret_clientSecret,
        "deletePrevSubmissions": deletePrevSubmissions,

        "sap_pipeline": sap_pipeline,
        "sap_pipelineType": sap_pipelineType,
        "ingestionHistoryMode": ingestionHistoryMode,
        "landingDateTimeFormat": landingDateTimeFormat 

    }

In [1]:
print(input_params)

In [ ]:
%run EdGraph/modules/SAP_PEIMS/v0.6/src/utilities/sap_peim_v0_6_sap_py

In [ ]:
from datetime import datetime
oea = EdFiOEAChild()   

In [ ]:
oea.set_workspace(workspace)

In [48]:
def execute_etl_step(condition,
                     nb_path,
                     nb_timeout,
                     nb_params,
                     nb_error,
                     etl_stage):
    global etl_status
    # FIXME: Parameterize etl_status 
    stage_start_time = datetime.now()
    if nb_error is None:
        etl_status = f"{etl_stage} - RUNNING"
        try:
            if condition:
                mssparkutils.notebook.run(path = nb_path,
                                          timeout_seconds = nb_timeout,
                                          arguments = nb_params)
                etl_status = f"{etl_stage} - SUCCEEDED"
            else:
                etl_status = f"{etl_stage} - SKIPPED"
        except Exception as error_msg:
            etl_status = f"{etl_stage} - FAILED"
            logger.info(f"{error_msg}")
            nb_error = error_msg
    else:
        pass
    
    stage_end_time = datetime.now()
    log_data = pipeline_error_logger.create_log_dict(uniqueId = pipeline_error_logger.generate_random_alphanumeric(10), # Generate a random 10-character alphanumeric value
                                            pipelineExecutionId = pipelineExecutionId,#'TEST_1234',#executionId,
                                            sparkSessionId = spark.sparkContext.applicationId,
                                            stageName = etl_stage,
                                            start_time = stage_start_time,
                                            end_time = stage_end_time,
                                            run_status = 'SUCCEEDED' if nb_error is None else 'FAILED',
                                            etl_status = etl_status,
                                            error_description = str(nb_error))
    pipeline_error_logger.consolidate_logs(log_data,'stage')
    if ' - SUCCEEDED' in etl_status or ' - SKIPPED' in etl_status:
        return etl_status, None
    else:
        return etl_status, nb_error

def get_entity_logs_status(df, executionId, stageName):
    df = df.filter((F.col('pipelineExecutionId') == executionId) & (F.col('stageName') == stageName))
    df = df.withColumn('numRecordsFailed', F.col('totalNumOutputRows') - F.col('numInputRows'))

    if df.filter(F.col("numRecordsFailed") == 0).count() == df.count():
        return 'success'
    
    if df.filter(F.col("numRecordsFailed") != 0).count() == df.count():
        return 'failure'
    
    if df.agg(F.sum("numRecordsFailed")).collect()[0][0] != 0:
        return 'partial-success'
    
    return 'unknown'

In [49]:
nb_root = "EdGraph/modules/SAP_PEIMS/v0.6/src"
nb_version = "sap_peim_v0_6"
nb_error = None

run_status = "INITIATED"
etl_status = ""

In [50]:
pipeline_error_logger = ErrorLogging(spark = spark, 
                                     oea = oea, 
                                     logger = logger)

try:
    run_status = "RUNNING"
    # NOTE: Prepare SAP Meta Data 
    etl_status, nb_error = execute_etl_step(condition = prepareSAPMetaData,
                                            nb_path = f"{nb_root}/utilities/{nb_version}_descriptors_dumping",
                                            nb_timeout = 1800,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Metadata-Preparation")
    # NOTE: Ingestion
    etl_status, nb_error = execute_etl_step(condition = etl_processing,
                                            nb_path = f"{nb_root}/main/{nb_version}_ingest",
                                            nb_timeout = 900,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Ingestion")
    # NOTE: Refinement
    etl_status, nb_error = execute_etl_step(condition = etl_processing or prepareSAPMetaData,
                                            nb_path = f"{nb_root}/main/{nb_version}_refine",
                                            nb_timeout = 7200 if prepareSAPMetaData else 1800,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Refinement")
    
    # NOTE: Move to Stage 3 (Submissions)
    etl_status, nb_error = execute_etl_step(condition = submissions,
                                            nb_path = f"{nb_root}/main/{nb_version}_migrate_to_s3",
                                            nb_timeout = 900,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Stage 3 Transfer (Submission)")

    # NOTE: Data Deletions
    etl_status, nb_error = execute_etl_step(condition = deletePrevSubmissions,
                                            nb_path = f"{nb_root}/main/{nb_version}_delete_from_edfi",
                                            nb_timeout = 2500,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "API-Deletion")

    # NOTE: Data Validation
    etl_status, nb_error = execute_etl_step(condition = validation,
                                            nb_path = f"{nb_root}/main/{nb_version}_create_views_for_validations",
                                            nb_timeout = 2500,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Submission")

    # NOTE: Data Submissions
    etl_status, nb_error = execute_etl_step(condition = submissions,
                                            nb_path = f"{nb_root}/main/{nb_version}_submit_to_edfi",
                                            nb_timeout = 3600,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "Submission")
    if nb_error is not None:
        raise Exception(f"{nb_error}")
    run_status = "SUCCEEDED"
except Exception as e:
    run_status = "FAILED"
    print(f'Pipeline Executed with Error {e}')

### Pipeline Level Logs

In [51]:
log_type = 'pipeline'

end_time = datetime.now()
log_data = pipeline_error_logger.create_log_dict(uniqueId = pipeline_error_logger.generate_random_alphanumeric(10), # Generate a random 10-character alphanumeric value
                                                pipelineExecutionId = pipelineExecutionId,#'TEST_1234',#executionId,
                                                sparkSessionId = spark.sparkContext.applicationId,
                                                start_time = start_time,
                                                end_time = end_time,
                                                run_status = run_status,
                                                etl_status = str(etl_status))
pipeline_error_logger.consolidate_logs(log_data,'pipeline')
df = pipeline_error_logger.create_spark_df('pipeline')


pipeline_error_logger.write_logs_to_delta_lake(df = df, 
                             log_type = 'pipeline',
                             destination_url = pipeline_error_logger.to_logs_url('etl-logs/log_type=pipeline'))
pipeline_error_logger.add_etl_logs_to_lake_db(db_name = f'ldb_{workspace}_sap_etl_logs',
                                     logs_base_path = 'etl-logs',
                                     log_type = 'pipeline',
                                     overwrite = True)

### Stage Level Logs

In [52]:
log_type = 'entity'
entity_logs_url = pipeline_error_logger.to_logs_url('etl-logs/log_type=entity')
entity_logs_df = oea.load(entity_logs_url)

In [53]:
df = pipeline_error_logger.create_spark_df('stage')
df = df.withColumn('stage_status', F.lit('Unknown'))

log_type = 'stage'
destination_url = pipeline_error_logger.to_logs_url('etl-logs/log_type=stage')

for stageName in ['Ingestion', 'Refinement']:
    stage_status = get_entity_logs_status(df = entity_logs_df, 
                                          executionId = pipelineExecutionId, 
                                          stageName = stageName)
                    
    df = df.withColumn('stage_status', 
        F.when((F.col('stageName') == stageName) & (F.col('pipelineExecutionId') == executionId), stage_status)
        .otherwise(F.col('stage_status'))
    )

pipeline_error_logger.write_logs_to_delta_lake(df = df, 
                                               log_type = log_type,
                                               destination_url = destination_url)
pipeline_error_logger.add_etl_logs_to_lake_db(db_name = f'ldb_{workspace}_sap_etl_logs',
                                     logs_base_path = 'etl-logs',
                                     log_type = 'stage',
                                     overwrite = True)